<a href="https://colab.research.google.com/github/Engineer-Ayesha-Shafique/Brain-Tumor-Segmentation-and-Detection-using-UNET-and-Watershed-in-Python/blob/main/Brain_Tumor_Segmentation_and_Detection_using_UNET_and_Watershed_in_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/drive' ,force_remount=True)

Mounted at /content/drive


#MOE




In [2]:
import os
BASE_MODEL_PATH = "/content/drive/MyDrive/Project_41725"
MODEL_1 = os.path.join(BASE_MODEL_PATH, "files" , "UNetModified" , "model.h5")
MODEL_2 = os.path.join(BASE_MODEL_PATH, "files" , "LinkNet" , "model.h5")
MODEL_3 = os.path.join(BASE_MODEL_PATH, "files", "aug_pass" , "model_512.h5")
MODEL_4 = os.path.join(BASE_MODEL_PATH, "files", "DG" , "model.h5")
MODEL_5 = os.path.join(BASE_MODEL_PATH, "files", "DG" , "model_256.h5")

In [3]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    print(f"✅ TensorFlow will use {len(gpus)} GPU(s)")
else:
    print("⚠️ No GPU detected for TensorFlow")


✅ TensorFlow will use 1 GPU(s)


In [4]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold
from xgboost import XGBClassifier
from tqdm import tqdm
import imageio

In [5]:
# ── 1) Imports & constants ─────────────────────────────────────────────────────

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import (
    Input, Conv2D, Activation, Lambda
)

INPUT_SHAPE = (224, 224, 3)
NUM_CLASSES = 5

In [6]:
!pip install --quiet segmentation-models tensorflow-addons


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 kB 4.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
inflect 7.5.0 requires typeguard>=4.0.1, but you have typeguard 2.13.3 which is incompatible.


In [7]:
import os
os.environ["SM_FRAMEWORK"] = "tf.keras"

import segmentation_models as sm
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import (
    Conv2D, BatchNormalization, Activation,
    MaxPool2D
)
from tensorflow.keras.applications import EfficientNetB4

sm.set_framework('tf.keras')

BASE_MODEL_PATH = "/content/drive/MyDrive/Project_41725"
MODEL_1_UNet = os.path.join(BASE_MODEL_PATH, "files" , "UNetModified" , "model.h5")
MODEL_2_LINKNET = os.path.join(BASE_MODEL_PATH, "files" ,"LinkNet", "modell.h5")
MODEL_3_AttentionUNet = os.path.join(BASE_MODEL_PATH, "files", "aug_pass", "model_512.h5")
MODEL_4_DGNet = MODEL_4
MODEL_5_Upp =  MODEL_5
INPUT_SHAPE = (512, 512, 3)
ds_path = "/content/drive/MyDrive/Project_41725/newdata/train_maskimg"
mask_dir = "/content/drive/MyDrive/Project_41725/newdata/train_maskimg/masks"


Segmentation Models: using `tf.keras` framework.


In [8]:
# In a notebook cell, before any cv2 imports:
!pip install --quiet opencv-python-headless
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

# 1) Import TF first and set memory growth (fallback to experimental)
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        try:
            tf.config.experimental.set_memory_growth(gpu, True)
        except Exception as e:
            print(f"[GPU] Could not set memory growth: {e}")
    print(f"[GPU] Detected {len(gpus)} GPU(s).")

# 2) Now import the rest
import numpy as np
import cv2
from glob import glob
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import (
    ModelCheckpoint, CSVLogger, ReduceLROnPlateau,
    EarlyStopping, TensorBoard
)
from tensorflow.keras.optimizers import Adam
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, KFold
from xgboost import XGBClassifier
from tqdm import tqdm
import imageio
""" Global parameters """
Height = 512
Width  = 512

def create_folder(path):
    if not os.path.exists(path):
        os.makedirs(path)

def load_dataset(path_dataset, val_ratio=0.2, test_ratio=0.1, random_state=42):
    img_dir  = os.path.join(path_dataset, "images")
    mask_dir = os.path.join(path_dataset, "masks")

    image_exts = ('.png', '.jpg', '.jpeg')
    images = sorted([
        os.path.join(img_dir, f)
        for f in os.listdir(img_dir)
        if f.lower().endswith(image_exts)
    ])


    mask_exts = ('.tif', '.tiff')
    masks = sorted([
        os.path.join(mask_dir, f)
        for f in os.listdir(mask_dir)
        if f.lower().endswith(mask_exts)
    ])

    if not images:
        raise RuntimeError(f"No images found in {img_dir}")
    if not masks:
        raise RuntimeError(f"No masks found in {mask_dir}")

    images_lookup = {
        os.path.splitext(os.path.basename(p))[0]: p
        for p in images
    }
#MildDemented_0a7b1321-eba0-40dc-85b8-de34241554a2_jpg.rf.70370fcd9ac0fb82f91d27f558053d84.tif
    pairs = []
    for m in masks:
        base = os.path.splitext(os.path.basename(m))[0]
        if base.endswith('_mask'):
            root = base[:-5]
            img_path = images_lookup.get(root)
            if img_path:
                pairs.append((img_path, m))

    if not pairs:
        raise RuntimeError("No matching image↔mask pairs found!")

    trainval, test = train_test_split(pairs, test_size=test_ratio, random_state=random_state)
    val_frac = val_ratio / (1.0 - test_ratio)
    train, val = train_test_split(trainval, test_size=val_frac, random_state=random_state)

    train_imgs, train_masks = zip(*train)
    val_imgs,   val_masks   = zip(*val)
    test_imgs,  test_masks  = zip(*test)

    return (
        list(train_imgs), list(train_masks),
        list(val_imgs),   list(val_masks),
        list(test_imgs),  list(test_masks)
    )

def read_image_file(path_img):
    path = path_img.decode('utf-8') if isinstance(path_img, bytes) else path_img
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    img = cv2.resize(img, (Width, Height)) / 255.0
    return img.astype(np.float32)

def read_mask_file(path_mask):
    path = path_mask.decode('utf-8') if isinstance(path_mask, bytes) else path_mask
    mask = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    if mask.ndim == 3:
        mask = mask[..., 0]
    mask = cv2.resize(mask, (Width, Height), interpolation=cv2.INTER_NEAREST)

    # ✅ Clip to [0, 4], not [0, 3]:
    mask = np.clip(mask, 0, NUM_CLASSES-1)

    # Convert to integer class IDs
    mask = mask.astype(np.int32)

    # Add the channel dim back
    return mask[..., None]

def tf_parse(path_img, path_mask):
    # Tell numpy_function: img→float32, mask→int32
    img, mask = tf.numpy_function(
        lambda pi, pm: (read_image_file(pi), read_mask_file(pm)),
        [path_img, path_mask],
        [tf.float32, tf.int32]          # <-- second output is now int32
    )
    img.set_shape([Height, Width, 3])
    mask.set_shape([Height, Width, 1])
    return img, mask

def tf_dataset(inputs, targets, batch_size=2):
    ds = tf.data.Dataset.from_tensor_slices((inputs, targets))
    ds = ds.map(tf_parse, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return ds

def edit_mask_images(mask_dir, func, output_dir=None):
    out_dir = output_dir or mask_dir
    create_folder(out_dir)
    exts = ('.tif')
    for fname in os.listdir(mask_dir):
        if not fname.lower().endswith(exts):
            continue
        in_path  = os.path.join(mask_dir, fname)
        mask     = cv2.imread(in_path, cv2.IMREAD_UNCHANGED)
        edited   = func(mask)
        cv2.imwrite(os.path.join(out_dir, fname), edited)
    print(f"[edit_mask_images] Applied edits to masks in {out_dir}")

if __name__ == "__main__":
    np.random.seed(42)
    tf.random.set_seed(42)

    create_folder("/content/drive/MyDrive/Project_41725/files")

    batch_size = 16
    lr         = 1e-4
    epochs_num     = 100
    path_model = os.path.join("/content/drive/MyDrive/Project_41725/files", "model.h5")
    path_csv   = os.path.join("/content/drive/MyDrive/Project_41725/files", "log.csv")

    ds_path = "/content/drive/MyDrive/Project_41725/newdata/train_maskimg"
    train_imgs, train_masks, val_imgs, val_masks, test_imgs, test_masks = load_dataset(ds_path)

    print(f"Train: {len(train_imgs)}, Val: {len(val_imgs)}, Test: {len(test_imgs)}")

    train_ds = tf_dataset(train_imgs, train_masks, batch_size=batch_size)
    val_ds   = tf_dataset(val_imgs,   val_masks,   batch_size=batch_size)
    test_ds  = tf_dataset(test_imgs,  test_masks,  batch_size=batch_size)

    # Build, compile and fit your model below:
    # model = Unet_model((Height, Width, 3))
    # model.compile(loss=dice_loss, optimizer=Adam(lr), metrics=[dice_coeff])
    # callbacks = [ModelCheckpoint(...), ...]
    # model.fit(train_ds, validation_data=val_ds, epochs=epochs, callbacks=callbacks)


[GPU] Detected 1 GPU(s).
Train: 1216, Val: 348, Test: 174


In [9]:
import pandas as pd

# assuming you’ve re-run the load_dataset cell in XGBoost_ensemble.ipynb
# so train_imgs, train_masks, test_imgs are available:

pd.DataFrame({'image_path': train_imgs}).to_csv('train_images.csv', index=False)
pd.DataFrame({'mask_path':  train_masks}).to_csv('train_masks.csv',  index=False)
pd.DataFrame({'image_path': test_imgs }).to_csv('test_images.csv',  index=False)
print("Manifest CSVs written.")

Manifest CSVs written.


In [10]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import (
    ModelCheckpoint, ReduceLROnPlateau, CSVLogger, EarlyStopping
)
import os
# Make sure segmentation_models picks up tf.keras, not plain keras
os.environ["SM_FRAMEWORK"] = "tf.keras"

import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import get_custom_objects
import segmentation_models as sm

# Hyper‑params & paths
NUM_CLASSES = 5
INPUT_SHAPE = (512, 512, 3)
EPOCHS      = 160
path_model  = "/content/drive/MyDrive/Project_41725/XGBoost/files/model.h5"
path_csv    = "/content/drive/MyDrive/Project_41725/XGBoost/files/log.csv"

# 0) kill any existing graph/state
tf.keras.backend.clear_session()

# 3) Define losses & metrics
def dice_coef(y_true, y_pred, smooth=1e-6):
    y_true = tf.squeeze(y_true, axis=-1)
    y_true_o = tf.one_hot(tf.cast(y_true, tf.int32), depth=NUM_CLASSES)
    y_true_f = tf.reshape(y_true_o, [-1, NUM_CLASSES])
    y_pred_f = tf.reshape(y_pred,   [-1, NUM_CLASSES])
    inter = tf.reduce_sum(y_true_f * y_pred_f, axis=0)
    denom = tf.reduce_sum(y_true_f, axis=0) + tf.reduce_sum(y_pred_f, axis=0)
    dice_per_class = (2. * inter + smooth) / (denom + smooth)
    return tf.reduce_mean(dice_per_class)

def dice_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

def combined_loss(y_true, y_pred):
    ce = tf.keras.losses.SparseCategoricalCrossentropy()(y_true, y_pred)
    return ce + dice_loss(y_true, y_pred)

class MeanIoUArgMax(tf.keras.metrics.MeanIoU):
    def __init__(self, num_classes=NUM_CLASSES, name="mean_iou", dtype=None):
        super().__init__(num_classes=num_classes, name=name, dtype=dtype)
    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.squeeze(y_true, axis=-1)
        y_pred = tf.argmax(y_pred, axis=-1)
        return super().update_state(y_true, y_pred, sample_weight)

for name, fn in sm.losses.__dict__.items():
    get_custom_objects()[name] = fn
for name, fn in sm.metrics.__dict__.items():
    get_custom_objects()[name] = fn
get_custom_objects().update({
    'dice_coef': dice_coef,
    'combined_loss': combined_loss,
    'MeanIoUArgMax': MeanIoUArgMax
})



In [11]:
!pip install --quiet mlxtend xgboost tqdm opencv-python-headless

In [12]:
import os
import numpy as np
import pandas as pd
import cv2
from tqdm import tqdm
from tensorflow.keras.models import load_model
from sklearn.model_selection import KFold
from xgboost import XGBClassifier

model1 = load_model(
    MODEL_1_UNet,
    compile=False
)
model2 = load_model(
    MODEL_2_LINKNET,
    compile=False
)
model3 = load_model(
    MODEL_3_AttentionUNet,
    compile=False
)
model4 = load_model(
    MODEL_4_DGNet,
    compile=False
)
model5 = load_model(
    MODEL_5_Upp,
    compile=False
)
model1 = tf.keras.models.load_model(MODEL_1_UNet, compile=False)
model2 = tf.keras.models.load_model(MODEL_2_LINKNET, compile=False)
model3 = tf.keras.models.load_model(MODEL_3_AttentionUNet, compile=False)
model4 = tf.keras.models.load_model(MODEL_4_DGNet, compile=False)
model5 = tf.keras.models.load_model(MODEL_5_Upp, compile=False)

In [19]:
import os
import cv2
import numpy as np
from glob import glob
from tensorflow.keras.models import load_model
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from xgboost import XGBClassifier
from tqdm.auto import tqdm
import joblib

# ─── CONFIG ────────────────────────────────────────────────────────────────
BASE_MODEL_DIR     = "/content/drive/MyDrive/Project_41725/files"
MODEL_DIR          = "/content/drive/MyDrive/Project_41725/models_new"
os.makedirs(MODEL_DIR, exist_ok=True)

MODEL_PATHS        = [
    "UNetModified/model.h5",
    "LinkNet/model.h5",
    "aug_pass/model_512.h5",
    "DG/model.h5",
    "DG/model_256.h5",
]
IMG_DIR            = "/content/drive/MyDrive/Project_41725/newdata/train_maskimg/images"
MSK_DIR            = "/content/drive/MyDrive/Project_41725/newdata/train_maskimg/masks"
IMG_EXTS           = (".png", ".jpg", ".jpeg")
MSK_EXTS           = (".tif", ".tiff")
SAMPLES_PER_IMAGE  = 200
MAX_IMAGES         = 300

# ─── LOAD BASE MODELS ─────────────────────────────────────────────────────
print("🔄 Loading base segmentation models…")
base_models = [
    load_model(os.path.join(BASE_MODEL_DIR, p), compile=False)
    for p in tqdm(MODEL_PATHS, desc="Base models")
]
NUM_CLASSES = base_models[0].output_shape[-1]
IN_H, IN_W  = base_models[0].input_shape[1:3]

# ─── PAIR UP IMAGES ↔ MASKS FOR LATER INJECTION ────────────────────────────
# assume mask file is named "<stem>_mask.tif" matching "<stem>.png"
images_lookup = {
    os.path.splitext(os.path.basename(p))[0]: p
    for ext in IMG_EXTS
    for p in glob(os.path.join(IMG_DIR, f"*{ext}"))
}
pairs = []
for ext in MSK_EXTS:
    for msk_path in glob(os.path.join(MSK_DIR, f"*{ext}")):
        stem = os.path.splitext(os.path.basename(msk_path))[0]
        if stem.endswith("_mask"):
            key = stem[:-5]
            img_p = images_lookup.get(key)
            if img_p:
                pairs.append((img_p, msk_path))

# ─── FEATURE EXTRACTION ───────────────────────────────────────────────────
def get_pixel_probs(img, models):
    feats = []
    for m in models:
        pm = m.predict(img[None], verbose=0)[0]               # (H,W,C)
        feats.append(pm.reshape(-1, NUM_CLASSES))            # (H*W,C)
    return np.concatenate(feats, axis=1)                     # (H*W, M*C)

# ─── SAMPLE TRAINING DATA ─────────────────────────────────────────────────
print("🔄 Sampling pixels & building training set…")
train_imgs  = sorted(pairs, key=lambda x: x[0])[:MAX_IMAGES]
X_chunks, y_chunks = [], []
for img_p, msk_p in tqdm(train_imgs, desc="Images"):
    img = cv2.imread(img_p)[..., ::-1] / 255.0
    msk = cv2.imread(msk_p, cv2.IMREAD_GRAYSCALE)
    img_rs = cv2.resize(img, (IN_W, IN_H), cv2.INTER_AREA)
    msk_rs = cv2.resize(msk, (IN_W, IN_H), cv2.INTER_NEAREST).ravel()
    P = get_pixel_probs(img_rs, base_models)               # (H*W, M*C)
    idxs = np.random.choice(IN_H*IN_W, SAMPLES_PER_IMAGE, replace=False)
    X_chunks.append(P[idxs]);  y_chunks.append(msk_rs[idxs])

X_train = np.vstack(X_chunks)
y_train = np.concatenate(y_chunks)
print(f"  → initial X_train: {X_train.shape}, y_train: {y_train.shape}")

# ─── INJECT MISSING CLASSES ────────────────────────────────────────────────
present = set(np.unique(y_train))
missing = set(range(NUM_CLASSES)) - present
if missing:
    print(f"⚠️ Missing classes: {missing}, injecting one pixel each…")
    for cls in missing:
        for img_p, msk_p in pairs:
            m_full = cv2.imread(msk_p, cv2.IMREAD_GRAYSCALE)
            ys, xs = np.where(m_full == cls)
            if len(ys):
                # take first occurrence
                y0, x0 = ys[0], xs[0]
                img_full = cv2.imread(img_p)[..., ::-1] / 255.0
                img_rs   = cv2.resize(img_full, (IN_W, IN_H), cv2.INTER_AREA)
                P        = get_pixel_probs(img_rs, base_models)
                # map full-coord → resized-coord
                yr = int(y0 * IN_H / m_full.shape[0])
                xr = int(x0 * IN_W / m_full.shape[1])
                feat     = P[yr*IN_W + xr]
                X_train  = np.vstack([X_train, feat])
                y_train  = np.hstack([y_train, cls])
                break
    print(f"  → now y_train covers classes: {np.unique(y_train)}")

# ─── TRAIN META-LEARNERS ───────────────────────────────────────────────────
print("🔄 Training RandomForest…")
rf = RandomForestClassifier(
    n_estimators=100,
    max_depth=10,
    n_jobs=-1,
    random_state=42
)
print("🔄 Training XGBoost…")
xgb = XGBClassifier(
    objective="multi:softprob",
    use_label_encoder=False,
    eval_metric="mlogloss",
    n_estimators=100,
    max_depth=3,
    tree_method="hist",
    random_state=42
)
print("⏳ Fitting RF…");  rf.fit(X_train, y_train)
print("⏳ Fitting XGB…"); xgb.fit(X_train, y_train)

# ─── BUILD SOFT-VOTING ENSEMBLE ───────────────────────────────────────────
print("🔄 Building soft-voting ensemble…")
ensemble = VotingClassifier(
    estimators=[("rf", rf), ("xgb", xgb)],
    voting="soft",
    n_jobs=-1
)
ensemble.fit(X_train, y_train)

# ─── SAVE ALL MODELS ───────────────────────────────────────────────────────
print("💾 Saving trained models…")
joblib.dump(rf,                     os.path.join(MODEL_DIR, "rf_meta.pkl"))
xgb.save_model(                     os.path.join(MODEL_DIR, "xgb_meta.model"))
joblib.dump(ensemble,               os.path.join(MODEL_DIR, "ensemble_soft_voting.pkl"))
print("✅ Models written to", MODEL_DIR)

# ─── UPDATED segment_image WITH PATH CHECK ─────────────────────────────────
import os
import cv2
import numpy as np

def segment_image(img_path, out_path="seg.png"):
    # 0) sanity check
    if not os.path.isfile(img_path):
        raise FileNotFoundError(f"segment_image: no such file → {img_path}")
    # 1) read & normalize
    img_bgr = cv2.imread(img_path)
    if img_bgr is None:
        raise ValueError(f"segment_image: failed to decode image → {img_path}")
    img = img_bgr[..., ::-1] / 255.0
    H, W = img.shape[:2]

    # 2) resize to model input
    img_rs = cv2.resize(img, (IN_W, IN_H), interpolation=cv2.INTER_AREA)

    # 3) feature extraction & ensemble predict
    P   = get_pixel_probs(img_rs, base_models)    # (IN_H*IN_W, M·C)
    lbl = ensemble.predict(P)                      # (IN_H*IN_W,)
    mask = lbl.reshape(IN_H, IN_W).astype(np.uint8)

    # 4) upsample & save
    mask_full = cv2.resize(mask, (W, H), cv2.INTER_NEAREST)
    cv2.imwrite(out_path, mask_full)
    print(f"🔍 Saved segmentation → {out_path}")
    return mask_full


# ─── USAGE ─────────────────────────────────────────────────────────────────
if __name__=="__main__":
    test_img = "/content/drive/MyDrive/Project_41725/newdata/train_maskimg/"
    _ = segment_image(test_img, out_path=os.path.join(MODEL_DIR,"ensemble_seg.png"))


🔄 Loading base segmentation models…


Base models:   0%|          | 0/5 [00:00<?, ?it/s]

🔄 Sampling pixels & building training set…


Images:   0%|          | 0/300 [00:00<?, ?it/s]

  → initial X_train: (60000, 25), y_train: (60000,)
🔄 Training RandomForest…
🔄 Training XGBoost…
⏳ Fitting RF…
⏳ Fitting XGB…


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [16:42:54] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


🔄 Building soft-voting ensemble…
💾 Saving trained models…
✅ Models written to /content/drive/MyDrive/Project_41725/models_new


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [16:43:02] WARNING: /workspace/src/c_api/c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)


FileNotFoundError: segment_image: no such file → /content/drive/MyDrive/Project_41725/newdata/test/image123.png

In [ ]:
# ─── 0) INSTALL & IMPORTS ──────────────────────────────────────────────────
!pip install --quiet opencv-python-headless xgboost tqdm

from google.colab import files
from IPython.display import display
import os, cv2, joblib, numpy as np, matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import BaseEstimator
from tqdm import tqdm

# ─── 1) CONFIG ─────────────────────────────────────────────────────────────
MODEL_DIR        = "/content/drive/MyDrive/Project_41725/models_new"
BASE_MODEL_DIR   = "/content/drive/MyDrive/Project_41725/files"
MODEL_PATHS      = [
    "UNetModified/model.h5",
    "LinkNet/model.h5",
    "aug_pass/model_512.h5",
    "DG/model.h5",
    "DG/model_256.h5",
]
# how many pixels *per class* per image to sample
SAMPLES_PER_CLASS_PER_IMAGE = 50

os.makedirs(MODEL_DIR, exist_ok=True)

# ─── 2) LOAD BASE SEGMENTATION MODELS ───────────────────────────────────────
print("Loading base segmentation models…")
base_models = []
for rel in tqdm(MODEL_PATHS, desc="Base models"):
    path = os.path.join(BASE_MODEL_DIR, rel)
    if not os.path.isfile(path):
        raise FileNotFoundError(f"Base model not found: {path}")
    base_models.append(load_model(path, compile=False))

# derive some handy constants
NUM_CLASSES = base_models[0].output_shape[-1]
IN_H, IN_W  = base_models[0].input_shape[1:3]
print(f"→ expecting {NUM_CLASSES} classes, input size {IN_H}×{IN_W}")

# ─── 3) FEATURE EXTRACTOR ───────────────────────────────────────────────────
def get_pixel_probs(img_rs):
    """
    img_rs: IN_H×IN_W×3 float32 in [0,1]
    returns: (IN_H*IN_W, M * NUM_CLASSES) array of softmax probabilities
    """
    ps = []
    for m in base_models:
        pm = m.predict(img_rs[None], verbose=0)[0]      # (IN_H,IN_W,NUM_CLASSES)
        ps.append(pm.reshape(-1, NUM_CLASSES))         # (IN_H*IN_W,NUM_CLASSES)
    return np.concatenate(ps, axis=1)                 # (IN_H*IN_W, M·C)

# ─── 4) BALANCED PIXEL SAMPLING & TRAIN SET BUILDING ───────────────────────
from collections import defaultdict

IMG_DIR  = "/content/drive/MyDrive/Project_41725/newdata/train_maskimg/images"
MSK_DIR  = "/content/drive/MyDrive/Project_41725/newdata/train_maskimg/masks"
IMG_EXTS = (".png", ".jpg", ".jpeg")
MSK_EXTS = (".tif", ".tiff")

train_imgs  = sorted(sum([list(tqdm.__wrapped__(glob := __import__('glob').glob(
    os.path.join(IMG_DIR, f"*{ext}")))) for ext in IMG_EXTS], []))
train_masks = sorted(sum([list(glob(os.path.join(MSK_DIR, f"*{ext}"))) for ext in MSK_EXTS], []))

if len(train_imgs) != len(train_masks):
    raise RuntimeError("Image/mask count mismatch!")

X_per_cls = defaultdict(list)
y_per_cls = defaultdict(list)

print("Sampling pixels per class…")
for img_path, msk_path in tqdm(zip(train_imgs, train_masks),
                               total=len(train_imgs),
                               desc="Images"):
    # load & prep
    img   = cv2.imread(img_path)[..., ::-1].astype(np.float32)/255.0
    msk   = cv2.imread(msk_path, cv2.IMREAD_GRAYSCALE)
    img_rs = cv2.resize(img, (IN_W, IN_H), interpolation=cv2.INTER_AREA)
    msk_rs = cv2.resize(msk, (IN_W, IN_H),
                        interpolation=cv2.INTER_NEAREST).ravel()
    P      = get_pixel_probs(img_rs)  # (IN_H*IN_W, M·C)

    for cls in range(NUM_CLASSES):
        idxs = np.where(msk_rs == cls)[0]
        if len(idxs)==0:
            continue
        take = np.random.choice(
            idxs,
            min(SAMPLES_PER_CLASS_PER_IMAGE, len(idxs)),
            replace=False
        )
        X_per_cls[cls].append(P[take])
        y_per_cls[cls].append(np.full(len(take), cls, dtype=np.int32))

# stack into final X_train, y_train
X_list, y_list = [], []
for cls in range(NUM_CLASSES):
    if X_per_cls[cls]:
        X_list.append(np.vstack(X_per_cls[cls]))
        y_list.append(np.concatenate(y_per_cls[cls]))

X_train = np.vstack(X_list)
y_train = np.concatenate(y_list)
print("  → X_train:", X_train.shape, "y_train classes:", np.unique(y_train))

# ─── 5) TRAIN META-LEARNERS & SAVE ─────────────────────────────────────────
print("Training RandomForest…")
rf = RandomForestClassifier(n_estimators=100,
                            n_jobs=-1,
                            random_state=42)
rf.fit(X_train, y_train)

print("Training XGBoost…")
xgb = XGBClassifier(
    objective="multi:softprob",
    num_class=NUM_CLASSES,
    eval_metric="mlogloss",
    use_label_encoder=False,
    n_jobs=4,
    random_state=42
)
xgb.fit(X_train, y_train)

# save them
rf_pth  = os.path.join(MODEL_DIR, "rf_meta.pkl")
xgb_pth = os.path.join(MODEL_DIR, "xgb_meta.ubj")
joblib.dump(rf, rf_pth)
xgb.save_model(xgb_pth)
print(f"✅ Meta-learners saved →\n  {rf_pth}\n  {xgb_pth}")

# ─── 6) MANUAL SOFT-VOTING WRAPPER ──────────────────────────────────────────
class ManualVotingClassifier(BaseEstimator):
    def __init__(self, estimators):
        self.models = [m for _,m in estimators]
    def predict_proba(self, X):
        ps = [m.predict_proba(X) for m in self.models]
        return sum(ps)/len(ps)
    def predict(self, X):
        return np.argmax(self.predict_proba(X), axis=1)

vc_meta = ManualVotingClassifier([
    ("rf", rf),
    ("xgb", xgb),
])

# ─── 7) INFERENCE FUNCTION ─────────────────────────────────────────────────
def segment_image(img_path, out_path="seg.png"):
    if not os.path.isfile(img_path):
        raise FileNotFoundError(img_path)
    bgr = cv2.imread(img_path)
    if bgr is None:
        raise ValueError(f"Bad image: {img_path}")
    rgb = bgr[..., ::-1].astype(np.float32)/255.0
    H,W=rgb.shape[:2]
    rs=cv2.resize(rgb,(IN_W,IN_H),cv2.INTER_AREA)
    P = get_pixel_probs(rs)        # (IN_H*IN_W, M·C)
    lbl = vc_meta.predict(P)       # (IN_H*IN_W,)
    mask=lbl.reshape(IN_H,IN_W).astype(np.uint8)
    full=cv2.resize(mask,(W,H),cv2.INTER_NEAREST)
    cv2.imwrite(out_path, full)
    return rgb, full

# ─── 8) UPLOAD & RUN GUI ───────────────────────────────────────────────────
print("▶ Upload one or more MRI slices…")
uploaded = files.upload()
for fn in uploaded.keys():
    print("→ Segmenting", fn)
    orig, seg = segment_image(fn, out_path=f"seg_{fn}")
    fig,axs=plt.subplots(1,2,figsize=(10,5))
    axs[0].imshow(orig); axs[0].set_title("Input"); axs[0].axis("off")
    axs[1].imshow(seg,cmap="gray"); axs[1].set_title("Mask"); axs[1].axis("off")
    plt.show()


In [24]:
# ─── INSTALL ────────────────────────────────────────────────────────────────
!pip install --quiet gradio

# ─── IMPORTS ────────────────────────────────────────────────────────────────
import gradio as gr
import cv2, os, numpy as np
from sklearn.base import BaseEstimator
from xgboost import XGBClassifier
import joblib
from tensorflow.keras.models import load_model

# ─── LOAD META-LEARNERS ─────────────────────────────────────────────────────
MODEL_DIR = "/content/drive/MyDrive/Project_41725/models_new"
xgb_meta  = XGBClassifier();      xgb_meta.load_model(os.path.join(MODEL_DIR, "xgb_meta.model"))
rf_meta   = joblib.load(os.path.join(MODEL_DIR, "rf_meta.pkl"))

class ManualVotingClassifier(BaseEstimator):
    def __init__(self, estimators):
        self.models = [m for _, m in estimators]
    def predict_proba(self, X):
        probs = [m.predict_proba(X) for m in self.models]
        return sum(probs) / len(probs)
    def predict(self, X):
        return np.argmax(self.predict_proba(X), axis=1)

vc_meta = ManualVotingClassifier([("xgb", xgb_meta), ("rf", rf_meta)])

# ─── LOAD BASE SEGMENTATION MODELS ───────────────────────────────────────────
BASE_MODEL_DIR = "/content/drive/MyDrive/Project_41725/files"
MODEL_PATHS = [
    "UNetModified/model.h5",
    "LinkNet/model.h5",
    "aug_pass/model_512.h5",
    "DG/model.h5",
    "DG/model_256.h5",
]
base_models = [ load_model(os.path.join(BASE_MODEL_DIR,p), compile=False)
                for p in MODEL_PATHS ]

NUM_CLASSES = base_models[0].output_shape[-1]
IN_H, IN_W  = base_models[0].input_shape[1:3]

# ─── FEATURE EXTRACTOR ───────────────────────────────────────────────────────
def get_pixel_probs(img_rs):
    """img_rs: IN_H×IN_W×3 float32 → (IN_H*IN_W, M*NUM_CLASSES)"""
    feats = []
    for m in base_models:
        pm = m.predict(img_rs[None], verbose=0)[0]        # (IN_H,IN_W,NUM_CLASSES)
        feats.append(pm.reshape(-1, NUM_CLASSES))
    return np.concatenate(feats, axis=1)

# ─── CORE INFERENCE ─────────────────────────────────────────────────────────
def segment_image_np(img_rgb):
    """
    img_rgb: H×W×3 in RGB uint8
    returns: H×W mask uint8
    """
    # normalize & resize
    img_f = img_rgb.astype(np.float32)/255.0
    H, W = img_f.shape[:2]
    img_rs = cv2.resize(img_f, (IN_W, IN_H), interpolation=cv2.INTER_AREA)
    P      = get_pixel_probs(img_rs)                 # (IN_H*IN_W, M·C)
    lbl    = vc_meta.predict(P)                      # (IN_H*IN_W,)
    mask_rs= lbl.reshape(IN_H, IN_W).astype(np.uint8)
    mask   = cv2.resize(mask_rs, (W, H), interpolation=cv2.INTER_NEAREST)
    return mask

# ─── GRADIO WRAPPER ─────────────────────────────────────────────────────────
def gradio_predict(in_img):
    # gradio gives H×W×3 in RGB
    mask = segment_image_np(in_img)
    return in_img, mask

iface = gr.Interface(
    fn=gradio_predict,
    inputs=gr.Image(type="numpy", label="Upload MRI (RGB)"),
    outputs=[
      gr.Image(type="numpy", label="Original"),
      gr.Image(type="numpy", label="Segmentation Mask")
    ],
    title="Brain-Tumor Segmentation Ensemble",
    description="Upload an MRI slice → get a 5-model + RF/XGB ensemble mask."
)

iface.launch(debug=True)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.0/54.0 MB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.6/322.6 kB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 108.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 6.0 MB/s eta 0:00:00
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://9c79382a62a4719137.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://9c79382a62a4719137.gradio.live


In [ ]:
# ─── INSTALL ────────────────────────────────────────────────────────────────
!pip install --quiet gradio opencv-python-headless

# ─── IMPORTS ────────────────────────────────────────────────────────────────
import gradio as gr
import cv2, os, numpy as np, joblib
from sklearn.base import BaseEstimator
from xgboost import XGBClassifier
from tensorflow.keras.models import load_model

# ─── META-LEARNERS ───────────────────────────────────────────────────────────
MODEL_DIR = "/content/drive/MyDrive/Project_41725/models_new"
xgb_meta  = XGBClassifier(); xgb_meta.load_model(os.path.join(MODEL_DIR, "xgb_meta.model"))
rf_meta   = joblib.load(os.path.join(MODEL_DIR, "rf_meta.pkl"))

class ManualVotingClassifier(BaseEstimator):
    def __init__(self, estimators):
        self.models = [m for _, m in estimators]
    def predict_proba(self, X):
        probs = [m.predict_proba(X) for m in self.models]
        return sum(probs) / len(probs)
    def predict(self, X):
        return np.argmax(self.predict_proba(X), axis=1)

vc_meta = ManualVotingClassifier([("xgb", xgb_meta), ("rf", rf_meta)])

# ─── BASE SEGMENTATION MODELS ────────────────────────────────────────────────
BASE_MODEL_DIR = "/content/drive/MyDrive/Project_41725/files"
MODEL_PATHS = [
    "UNetModified/model.h5",
    "LinkNet/model.h5",
    "aug_pass/model_512.h5",
    "DG/model.h5",
    "DG/model_256.h5",
]
base_models = [
    load_model(os.path.join(BASE_MODEL_DIR, p), compile=False)
    for p in MODEL_PATHS
]
NUM_CLASSES = base_models[0].output_shape[-1]
IN_H, IN_W  = base_models[0].input_shape[1:3]

# ─── CLASS NAMES & PALETTE ──────────────────────────────────────────────────
class_names = {
    0: "background",
    1: "glioma",
    2: "mengoma",
    3: "pituitary",
    4: "dementia"
}
palette = {
    0: (  0,   0,   0),  # black
    1: (255,   0,   0),  # red
    2: (  0, 255,   0),  # green
    3: (  0,   0, 255),  # blue
    4: (255, 255,   0)   # yellow
}

# ─── FEATURE EXTRACTOR ───────────────────────────────────────────────────────
def get_pixel_probs(img_rs):
    feats = []
    for m in base_models:
        pm = m.predict(img_rs[None], verbose=0)[0]         # (H,W,C)
        feats.append(pm.reshape(-1, NUM_CLASSES))          # (H*W,C)
    return np.concatenate(feats, axis=1)                  # (H*W, M·C)

# ─── SEGMENT & STATS ────────────────────────────────────────────────────────
def segment_and_stats(img_rgb):
    # normalize & resize
    img_f  = img_rgb.astype(np.float32) / 255.0
    H, W   = img_f.shape[:2]
    img_rs = cv2.resize(img_f, (IN_W, IN_H), interpolation=cv2.INTER_AREA)

    # predict per-pixel class
    P      = get_pixel_probs(img_rs)
    lbl_rs = vc_meta.predict(P).reshape(IN_H, IN_W).astype(np.uint8)

    # upsample to original size
    mask   = cv2.resize(lbl_rs, (W, H), interpolation=cv2.INTER_NEAREST)

    # colorize mask
    color_mask = np.zeros((H, W, 3), dtype=np.uint8)
    for cls, color in palette.items():
        color_mask[mask == cls] = color

    # build distribution string
    unique, counts = np.unique(mask, return_counts=True)
    total = mask.size
    lines = []
    for u, c in zip(unique, counts):
        pct = c/total*100
        lines.append(f"{u} ({class_names[u]}): {c} px — {pct:.2f}%")
    dist_str = "\n".join(lines)

    # build HTML legend
    legend_items = []
    for cls, name in class_names.items():
        r,g,b = palette[cls]
        hexc = f"#{r:02x}{g:02x}{b:02x}"
        legend_items.append(
            f'<div style="display:inline-block;margin:4px">'+
            f'<div style="width:20px;height:20px;background:{hexc};display:inline-block;"></div> '+
            f'<span>Class {cls}: {name}</span></div>'
        )
    legend_html = "<div>" + "".join(legend_items) + "</div>"

    return color_mask, dist_str, legend_html

# ─── GRADIO INTERFACE ───────────────────────────────────────────────────────
def gradio_predict(inp_img):
    mask_color, stats, legend = segment_and_stats(inp_img)
    return inp_img, mask_color, stats, legend

iface = gr.Interface(
    fn=gradio_predict,
    inputs=gr.Image(type="numpy", label="Upload MRI (RGB)"),
    outputs=[
        gr.Image(type="numpy", label="Original"),
        gr.Image(type="numpy", label="Segmentation (color)"),
        gr.Textbox(label="Per-Class Pixel Distribution"),
        gr.HTML(label="Legend")
    ],
    title="Brain-Tumor Segmentation Ensemble",
    description="Drop in an MRI slice to get a colorized 5-class mask, per-class stats, and legend."
)

iface.launch(debug=True)


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://f846cc9e37eed5af02.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://f846cc9e37eed5af02.gradio.live
